In [ ]:
from mlflow import MlflowClient
client = MlflowClient()
client.create_model_version(
        name="LD_Image_Detection",
        source=<"your-model-source-path">,
        run_id=<"your-run-id">
    )

In [ ]:
import requests
%matplotlib inline
import matplotlib.pyplot as plt
import json
from PIL import Image
from io import BytesIO
import requests

subscription_key = "13b723adaded47b2bb95d90756692437"
analyze_url = "https://ld-image-detection.cognitiveservices.azure.com//vision/v3.1/analyze"
image_url = "https://www.pwc.com/jg/en/about-us/assets/jtmoty/jtmoty-08.jpg"
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params = {'visualFeatures':'Categories,Description,Objects'}
data = {'url': image_url}

try:
    response = requests.post(analyze_url,headers = headers, params=params, json = data)
    response.raise_for_status()
    analysis = response.json()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

#display
image = Image.open(BytesIO(requests.get(image_url).content))
plt.show(image)
plt.axis("off")
plt.show()
print(analysis)

In [ ]:
analysis['objects']

In [ ]:
import numpy as np
import cv2

def generate_box (image_url, boundRect):
    image = Image.open(BytesIO(requests.get(image_url).content))
    np_img = np.array(image)
    drawing = np_img
    for i in range(len(boundRect)):
        color = (255,0,0)
        cv2.rectangle(drawing, (int(boundRect[i][0]),int(boundRect[i][1])),(int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])),color,2)

        # show image
        plt.figure(figsize=(14,8))
        plt.imshow(drawing)
        plt.axis("off")
        plt.show()

generate_box(image_url,objects)

In [ ]:
# import library

!pip install opencv-python-headless

import requests
import matplotlib.pyplot as plt
import json
import io
from PIL import Image
from io import BytesIO
import requests
import numpy as np
import cv2
import base64

############################################################
#IMAGE DETECTION#
#
# Functions
#
#
# object detection function
text = []
def obj_dim (analysis):
    objects = []
    for rec in analysis['objects']:
        k = []
        k.append(rec['rectangle']['x'])
        k.append(rec['rectangle']['y'])
        k.append(rec['rectangle']['w'])
        k.append(rec['rectangle']['h'])
        objects.append(k)
    return objects
#
#
# generate red box around the detected object
def generate_box (image_path, boundRect):
    _image = Image.open(image_path)
    np_img = np.array(_image)
    drawing = np_img
    for i in range(len(boundRect)):
        color = (255,0,0)
        cv2.rectangle(drawing, (int(boundRect[i][0]),int(boundRect[i][1])),(int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])),color,2)
        plt.figure(figsize=(14,8))
        plt.imshow(drawing)
        plt.axis("off")
        plt.show()
#   
#
# generate a function to crop images
def img_crop (image_path,image_dim):
    image = Image.open(image_path)
    cropped_img = []
    for i in range(len(image_dim)):
        x,y,w,h = image_dim[i]
        img = image.crop((x,y,x+w,y+h))
        cropped_img.append(img)
    return cropped_img

#
#
# recursive function to identity object

def rec_obj (image):
    image_path = image
    # Credentials for Azure AI services
    subscription_key = "13b723adaded47b2bb95d90756692437"
    analyze_url = "https://ld-image-detection.cognitiveservices.azure.com//vision/v3.1/analyze"
    headers = {'Ocp-Apim-Subscription-Key': subscription_key}

    # parameters for ML
    params = {'visualFeatures':'Categories,Description,Objects'}

    files = {'image':open(image_path,'rb')}
    print(files)
    error = ""
    # Run model on the image and return the params in json format
    try:
        response = requests.post(analyze_url,headers = headers, params=params, files=files)
        response.raise_for_status()
        analysis = response.json()
        # return analysis
        print(analysis)
        text.append(analysis['description']['captions'])
        # If analysis of the image detects object
        if analysis['objects'] != []:
            print('Image Detected')

            # call obj dim store fucntion
            objects = obj_dim(analysis)

            # call generate red box function
            generate_box(image_path,objects)

            # # create a temp folder to store the cropped images
            cropped_img_arr = img_crop(image_path,objects)
            if cropped_img_arr != []:
                 for i in range(len(cropped_img_arr)):
                     cropped_img_arr[i].save("/lakehouse/default/Files/Image Detection/Images/img0"+str(i)+".jpg")
                     rec_obj("/lakehouse/default/Files/Image Detection/Images/img0"+str(i)+".jpg")
        else:
            print("")
            #break       
    except requests.HTTPError as e:
        # Handle the HTTPError here
        error = str(f"HTTP Error: {e}")
        

    print(error)  
   
rec_obj("/lakehouse/default/Files/Image Detection/Image.jpg")
print(text)
# print(output)